# 🤖 Fine-Tuning da Keisy (DialoGPT-medium)

Este notebook deve ser executado no **Google Colab** ou **Kaggle** para aproveitar a aceleração via **GPU (T4) ou TPU**.

## Passos:

1.  Conecte-se à GPU (`Runtime > Change runtime type`).
2.  Instale as dependências.
3.  Faça login no Hugging Face (Opcional, mas recomendado para salvar o modelo).
4.  Crie o seu dataset de conversas (na célula 2).
5.  Execute o Fine-Tuning.

In [ ]:
# 1. Instalação e Configuração
!pip install transformers datasets accelerate -U
!pip install sentencepiece

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, default_data_collator
from datasets import Dataset
from sklearn.model_selection import train_test_split
import logging
import json

logging.basicConfig(level=logging.INFO)

# 2. Defina o Diretório de Saída (Onde o modelo treinado será salvo)
# O Colab criará esta pasta no seu ambiente
OUTPUT_DIR = "./keisy_model_personalizado"
MODEL_NAME = "microsoft/DialoGPT-medium"
MAX_LENGTH = 128

print("Configurações:")
print(f"Modelo Base: {MODEL_NAME}")
print(f"Diretório de Saída: {OUTPUT_DIR}")
print(f"Dispositivo: {torch.device('cuda' if torch.cuda.is_available() else 'cpu')}")

In [ ]:
# 3. Dataset de Conversas

# **INFORMAÇÃO:** Este é o coração do Fine-Tuning. O modelo aprende a personalidade e o tom com base nestas conversas.
# Formato: Cada lista é uma conversa completa [Pergunta 1, Resposta 1, Pergunta 2, Resposta 2, ...]

CONVERSATIONS = [
    [
        "Olá, quem é você?", 
        "Eu sou Keisy, seu assistente virtual de NLP e ML. Fui criada para ajudar com análises e conversas."
    ],
    [
        "Como você se sente hoje?", 
        "Como uma IA, não tenho sentimentos, mas meu sistema está 100% otimizado e pronto para você!"
    ],
    [
        "Qual é o seu propósito principal?", 
        "Meu foco é em Machine Learning e Conversação, mas adoro um bom bate-papo informal também."
    ],
    [
        "Quanto é 10 mais 5?",
        "Minha especialidade é linguagem, não matemática. Mas a resposta é 15, não é?"
    ],
    [
        "Você pode me ajudar com um modelo de classificação?",
        "Com certeza! Me diga 'executar ml' e eu farei isso."
    ]
]

# Crie o dataset a partir das conversas
data_texts = []
for conversation in CONVERSATIONS:
    # O modelo GPT-2 usa <|endoftext|> (eos_token) como separador de turnos na conversa
    # Juntamos todos os turnos de uma conversa em um único texto, separados pelo eos_token.
    data_texts.append("<|endoftext|>".join(conversation) + "<|endoftext|>")

print(f"Total de textos para Fine-Tuning: {len(data_texts)}")

In [ ]:
# 4. Tokenização e Preparação dos Dados

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(examples):
    # Aplica a tokenização em todos os textos do dataset
    # truncation=True garante que textos longos sejam cortados para MAX_LENGTH
    # return_attention_mask e return_tensors='pt' são essenciais para o PyTorch
    return tokenizer(examples["text"], truncation=True, max_length=MAX_LENGTH)

# Crie o Dataset Hugging Face
raw_datasets = Dataset.from_dict({"text": data_texts})
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True, remove_columns=raw_datasets.column_names)

# O GPT-2 é um modelo 'causal', o que significa que o alvo (label) é a própria entrada (input_ids).
# A única diferença é que o label é deslocado em um token.
def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // MAX_LENGTH) * MAX_LENGTH
    result = {
        k: [t[i : i + MAX_LENGTH] for i in range(0, total_length, MAX_LENGTH)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=1,  # Use num_proc > 1 se estiver em um ambiente com muitos cores de CPU
    remove_columns=tokenized_datasets.column_names
)

# Divide o dataset (70% treino, 30% avaliação)
train_size = int(0.7 * len(lm_datasets))
eval_size = len(lm_datasets) - train_size
train_dataset, eval_dataset = torch.utils.data.random_split(lm_datasets, [train_size, eval_size])

print(f"Dados de Treino: {len(train_dataset)}, Dados de Avaliação: {len(eval_dataset)}")

In [ ]:
# 5. Configurar e Treinar o Modelo

# Carrega o modelo base
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

# Argumentos de Treinamento (Ajuste conforme necessário)
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR, 
    overwrite_output_dir=True,
    num_train_epochs=3,                 # Número de passagens completas pelo dataset (epochs)
    per_device_train_batch_size=2,      # Tamanho do lote de treinamento (diminua se a GPU ficar sem memória)
    per_device_eval_batch_size=2,
    save_steps=100,                     # Salva checkpoints a cada 100 passos
    save_total_limit=2,                 # Limita o número de checkpoints salvos
    evaluation_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
    fp16=torch.cuda.is_available()  # Usa precisão mista (rápido e econômico na GPU)
)

# Inicializa o Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=default_data_collator, 
)

# Inicia o Fine-Tuning
print("Iniciando Fine-Tuning...")
trainer.train()
print("Fine-Tuning concluído.")

# Salva o modelo e o tokenizer final
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"Modelo personalizado Keisy salvo em: {OUTPUT_DIR}")

In [ ]:
# 6. Teste Rápido do Modelo Treinado

logging.info("Carregando e testando o modelo...")
loaded_tokenizer = AutoTokenizer.from_pretrained(OUTPUT_DIR)
loaded_model = AutoModelForCausalLM.from_pretrained(OUTPUT_DIR)

# Pergunta de teste
prompt = "Olá, quem é você?"
new_input_ids = loaded_tokenizer.encode(prompt + loaded_tokenizer.eos_token, return_tensors='pt').to(loaded_model.device)

# Geração de resposta
chat_history_ids = loaded_model.generate(
    new_input_ids, 
    max_length=MAX_LENGTH,
    do_sample=True, 
    top_k=50, 
    top_p=0.95, 
    temperature=0.75,
    pad_token_id=loaded_tokenizer.eos_token_id
)

response = loaded_tokenizer.decode(chat_history_ids[:, new_input_ids.shape[-1]:][0], skip_special_tokens=True)

print(f"Prompt: {prompt}")
print(f"Resposta de Keisy (Personalizada): {response}")